In [ ]:
# Author: Ehud Behar
# All macros in one def keyword
import os
import glob 
import pandas as pd
import numpy as np

In [6]:
def myfunc(directory):
    os.chdir(os.path.join(r"C:\Users\ehud\Documents\bell-pepper\2025-04-0609", directory)) # Set the current working directory
    print("Current working directory is\n", os.getcwd())
    csv_files=glob.glob("*.csv")  # Find all .csv files in the current working directory
    # from each csv file in the current working directory, read only the "Intensity" column and concatenate all columns to one dataframe, which we shall call `dataset`:
    dataset=pd.concat(
        [
            pd.read_csv(file, usecols=["Intensity"]).rename(
                columns={"Intensity": os.path.splitext(file)[0]}
            )
            for file in csv_files
        ],
        axis=1
    )
    # prepend to this dataframe the wavelength column and reset the result. The wv column is taken from one of the csv files that has already been read to the above dataframe.
    # All wv columns in this directory should be identical; For convenience we take the column from the first file.

    # Read the Wavelength column from the first csv file
    wavelength = pd.read_csv(csv_files[0], usecols=["Wavelength"])


    dataset=pd.concat([pd.read_csv(csv_files[0], usecols=["Wavelength"]).rename(columns={"Wavelength":"wv"}), dataset], axis=1)
    #return dataset
    # Create a new dataframe with the value of numerical integral of spectral intensity vs. wavelength, for each column:
    results = {}

    for col in range(dataset.shape[1] - 1):
        col_name = dataset.columns[col + 1]
        integral = np.trapezoid(dataset.iloc[:, col + 1], dataset['wv'])
        results[col_name] = integral
    # Convert the results to a DataFrame if needed
    integrals = pd.DataFrame([results])
    del results
    del col_name
    del integral
    print(integrals)
    # check that `openpyxl` is installed (pandas engine used to read xls files.
    # pip install openpyxl
    # read the correspondence file that matches measurement and its exposure time:
    deltat = pd.read_excel(
        r"..\correspondence_exposure_times.xlsx",
        sheet_name=directory,
        dtype={"file name": str}  # Specify the column to be read as string
    ).iloc[:, -2:] # import only the last two columns in the excel sheet
    deltat
    # Add a new row to the integrals dataset using values from the times column in `deltat`, matched by column names in `integrals` and filenames in deltat. 
    # Match the exposure times from deltat with the column names in integrals

    ##### This cell doesn't work for some reason

    # Match the exposure times from deltat with the column names in integrals
    exposure_times = deltat.set_index('file name')['exposure time (s)']

    # Add the exposure times as a new row to the integrals DataFrame
    integrals.loc[1] = exposure_times.values
    
    # Divide the first row (propto power) by the second row (exposure time) to get a measure for the energy arriving to the sensor
    integrals.loc[2] = integrals.loc[0] / integrals.loc[1]
    print(integrals)
    # Calculate the mean of the control group (mean of the last row in `integrals`)
    average_energy = integrals.loc[2].mean()
    # Write the result to a text file in the current working directory
    with open('..\\average_energy.txt', 'a') as f:
        f.write(f"{directory}\t{average_energy}\n")
    
myfunc("stage3")

Current working directory is
 C:\Users\ehud\Documents\bell-pepper\2025-04-0609\stage3
            001           002           003           004           005  \
0  7.409123e+06  7.015312e+06  7.885222e+06  6.739168e+06  6.088904e+06   

            006           007           008           009           010  ...  \
0  7.115420e+06  8.090240e+06  6.977859e+06  6.564119e+06  7.390417e+06  ...   

            111           112           113           114           115  \
0  6.465222e+06  6.811443e+06  6.308446e+06  6.455948e+06  6.687574e+06   

            116           117           118           119           120  
0  7.827063e+06  6.820505e+06  7.250141e+06  6.730236e+06  6.577190e+06  

[1 rows x 120 columns]
            001           002           003           004           005  \
0  7.409123e+06  7.015312e+06  7.885222e+06  6.739168e+06  6.088904e+06   
1  1.200000e+00  8.500000e-01  7.000000e-01  6.000000e-01  6.000000e-01   
2  6.174269e+06  8.253308e+06  1.126460e+07  1.123195e